# Inspect Detectron2 training on plankton dataset

In [ ]:
import os
import glob
import pickle
import json
import cv2
import random
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import time

from detectron2 import model_zoo
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2.data.transforms as T

import lib.training_functions as training_functions # Custom training functions
import lib.my_visualizer as my_visualizer # Custom Detectron2 visualizer to increase font size

random.seed(12)

### Choose output directory

List all output directories

In [ ]:
# Output directory
output_dir = 'output/output_*'
output_dirs = glob.glob(output_dir)
output_dirs.sort()
output_dirs

By default, select the last one

In [ ]:
output_dir = output_dirs[-1]

### Read training settings

In [ ]:
with open(os.path.join(output_dir, 'settings.pickle'),'rb') as set_file:
    settings = pickle.load(set_file)
settings    

### Prepare dataset

In [ ]:
data_dir = settings['dataset']
# Register the dataset to detectron2
for d in ['train', 'valid', 'test']:
    DatasetCatalog.register('plankton_' + d, lambda d=d: training_functions.my_dataset_function(os.path.join(data_dir, d)))
    MetadataCatalog.get('plankton_' + d).set(thing_classes=['plankton'])
plankton_metadata = MetadataCatalog.get('plankton_train')

### Have a look at a few training frames

In [ ]:
dataset_dicts = training_functions.format_bbox(os.path.join(data_dir, 'train')) 
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d['file_name'])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plankton_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize = (10,10))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

## Training

### Read training metrics

In [ ]:
training_metrics = []
with open(os.path.join(output_dir, 'metrics.json'), 'r') as f:
    for line in f:
        training_metrics.append(json.loads(line))

### Plot training and validation loss evolution

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(
    [m['iteration'] for m in training_metrics if 'total_loss' in m], 
    [m['total_loss'] for m in training_metrics if 'total_loss' in m]
)
plt.plot(
    [m['iteration'] for m in training_metrics if 'validation_loss' in m], 
    [m['validation_loss'] for m in training_metrics if 'validation_loss' in m])
plt.legend(['total_loss', 'validation_loss'], loc='best')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Iteration')
plt.ylim([min(plt.ylim()),1])
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(
    [m['iteration'] for m in training_metrics if 'total_loss' in m], 
    [m['total_loss'] for m in training_metrics if 'total_loss' in m]
)
plt.plot(
    [m['iteration'] for m in training_metrics if 'validation_loss' in m], 
    [m['validation_loss'] for m in training_metrics if 'validation_loss' in m])
plt.legend(['total_loss', 'validation_loss'], loc='best')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Iteration')
plt.ylim([min(plt.ylim()),0.45])
plt.show()

## Testing

### Look at test metrics

In [ ]:
with open(os.path.join(output_dir, 'test_results.pickle'),'rb') as results_file:
    results = pickle.load(results_file)
results

### Precision x recall curve

In [ ]:
from IPython.display import Image
Image(filename=os.path.join(output_dir, 'precision_recall.png')) 

### Have a look at a few predicted frames

Load model with default predictor

In [ ]:
# Get default config
cfg = get_cfg()
# Load training config
cfg.merge_from_file(os.path.join(output_dir, 'my_cfg.yaml'))
# Set detection threshold (can be played with)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = settings['test_threshold']
# Load model weights
cfg.MODEL.WEIGHTS = os.path.join(output_dir, 'model_final.pth')

predictor = DefaultPredictor(cfg)

Run prediction

In [ ]:
test_dataset_dicts = training_functions.format_bbox(os.path.join(data_dir, 'test'))
sample_test = random.sample(test_dataset_dicts, 10)
for d in sample_test:
    im = cv2.imread(d['file_name'])
    outputs = predictor(im)
    v = my_visualizer.Visualizer(im[:, :, ::-1],
                   metadata=plankton_metadata, 
                   scale=1
    )
    v = v.draw_instance_predictions(outputs['instances'].to('cpu'))
    plt.figure(figsize=(10,10))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.show()

Use single image predictor to predict 100 frames to see how it compares with batch prediction

In [ ]:
single_preds = {
    'img_name': [],
    'scores': [],
}

large_sample_test = random.sample(test_dataset_dicts, 100)
for d in large_sample_test:
    im = cv2.imread(d['file_name'])
    outputs = predictor(im)
    instances = outputs['instances']
    if len(instances) > 0:

        single_preds['img_name'].extend([d['file_name']] * len(instances))
        single_preds['scores'].extend(instances.scores.cpu().numpy().tolist())
single_preds = pd.DataFrame(single_preds)


## Run batch inference on same frames

Create predictor for batch inference

In [ ]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# Get default config
cfg = get_cfg()
# Load training config
cfg.merge_from_file(os.path.join(output_dir, 'my_cfg.yaml'))
# Set detection threshold (can be played with)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = settings['test_threshold']
# Build model 
batch_predictor = build_model(cfg)
# Load weights from training
DetectionCheckpointer(batch_predictor).load(os.path.join(output_dir, 'model_final.pth'))
# Change mode to eval
batch_predictor.eval()

Prepare batch of frames

In [ ]:
# Initiate empty list for frames
frames = []

for d in sample_test:
    # Read image
    im = cv2.imread(d['file_name'])
    height, width = im.shape[:2]
    
    # Resize image
    aug1 = T.ResizeShortestEdge(short_edge_length=[800], max_size=1980, sample_style='choice')
    frame = aug1.get_transform(im).apply_image(im)

    # Reshape image from (H, W, C) to (C, H, W) for Detectron2 input and convert to tensol 
    frame = torch.as_tensor(frame.astype('uint8').transpose(2, 0, 1))

    # Store in list of dicts with 'image', 'height' and 'width' entries
    frames.append({'image': frame, 'height': height, 'width': width})

Predict batch of frames

In [ ]:
with torch.no_grad():
    preds = batch_predictor(frames)

Extract and plot predictions

In [ ]:
for i in range(len(sample_test)):
    # Get frame    
    im = frames[i]['image'].numpy()
    # Reshape it
    im = np.moveaxis(im, 0, 2)
    # Resize it
    im = cv2.resize(im, preds[i]['instances'].to('cpu').image_size, interpolation = cv2.INTER_LINEAR) 
    
    # Plot predictions
    v = my_visualizer.Visualizer(im[:, :, ::-1],
                   metadata=plankton_metadata, 
                   scale=1
    )
    v = v.draw_instance_predictions(preds[i]['instances'].to('cpu'))
    plt.figure(figsize=(10,10))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
# Initiate empty list for frames
frames = []

for d in large_sample_test:
    # Read image
    im = cv2.imread(d['file_name'])
    height, width = im.shape[:2]
    
    # Resize image
    aug1 = T.ResizeShortestEdge(short_edge_length=[800], max_size=1980, sample_style='choice')
    frame = aug1.get_transform(im).apply_image(im)

    # Reshape image from (H, W, C) to (C, H, W) for Detectron2 input and convert to tensol 
    frame = torch.as_tensor(frame.astype('uint8').transpose(2, 0, 1))

    # Store in list of dicts with 'image', 'height' and 'width' entries
    frames.append({'image': frame, 'height': height, 'width': width})
    
# Predict frames    
with torch.no_grad():
    preds = batch_predictor(frames)

Inspect batch prediction of frames

In [ ]:
batch_preds = {
    'img_name': [],
    'scores': [],
}

for d, pred in zip(large_sample_test, preds):
    instances = pred['instances']
    #break
    if len(instances) > 0:
        batch_preds['img_name'].extend([d['file_name']] * len(instances))
        batch_preds['scores'].extend(instances.scores.cpu().numpy().tolist())
batch_preds = pd.DataFrame(batch_preds)

In [ ]:
print('Number of predictions on the same 100 frames:')
print(f'Single image predictor: {len(single_preds)}')
print(f'Batch predictor: {len(batch_preds)}')